In [1]:
import numpy as np

## A.1 日時

- 日付`NDay`は1/1からの通しの日数
- 時刻`NHour`は整数, 0時～23時が基本。12/31のみ24時あり
- 時刻`TT`は、時間分割`MM`における時刻
  - `MM`：1時間の内の$(1/$ `NDT`$)$間隔の順番, 正時が`MM`$=0$, `MM`$=0～$ `NDT`$-1$


- 時刻`Hour01`は、前時刻の`MM` $=$ `NDT`$/2～$同時刻の`MM` $=$ `NDT`$/2-1$の時間分割を、その正時に属するものとして扱うための時刻 → 時間毎の効果係数を算定するために使用するだけ

In [2]:
def calc_NDayNHour(Hour00):
    if Hour00==8760:
        [NDay, NHour] = [365, 24]
    else:
        NDay = Hour00//24 + 1
        NHour = Hour00 - (NDay - 1) * 24 

    return [NDay,NHour]

In [3]:
def calc_TT(NHour, NDT, MM):
    TT = NHour + MM / float(NDT)
    return TT

In [4]:
def calc_Hour01(TT):
    Hour01 = int(TT + 0.5)
    return Hour01

## A.2 赤緯の計算 (仕様書6.2 式(4))

- 赤緯$\delta_d [deg]$, $N$: 1月1日を$N=1$とした年頭からの通しの日数$[day]$
  - 右辺の余弦のかっこ内の角度は$radian$単位となっているので注意
  
$$ \begin{align}
\delta_d = (180 / \pi) & \{0.006322 - 0.405748 \cos (2 \pi N / 366 + 0.153231)\\
& - 0.005880 \cos (4 \pi N / 366 + 0.207099)\\
& - 0.003233 \cos (6 \pi N / 366 + 0.620129) \} \qquad \qquad \qquad (4) \\
\end{align} $$

In [5]:
def calc_deltad(NDay):
    deltad = (180 / np.pi) * (0.006322 - 0.405748 * np.cos(2 * np.pi * float(NDay) / 366 + 0.153231)
                                       - 0.005880 * np.cos(4 * np.pi * float(NDay) / 366 + 0.207099)
                                       - 0.003233 * np.cos(6 * np.pi * float(NDay) / 366 + 0.620129))
    
    return deltad

## A.3 均時差の計算 (仕様書6.2 式(6))

- 均時差$e_d[hour]$, $N$: 1月1日を$N=1$とした年頭からの通しの日数$[day]$
  - 右辺の余弦のかっこ内の角度は$radian$単位となっているので注意

$$ \begin{align}
e_d = -0.000279 &+ 0.122772 \cos (2 \pi N / 366 + 1.498311)\\
& - 0.165458 \cos (4 \pi N / 366 - 1.261546)\\
& - 0.005354 \cos (6 \pi N / 366 - 1.1571) \} \qquad \qquad \qquad (6) \\
\end{align} $$

In [6]:
def calc_eed(NDay):
    eed = ( -0.000279 + 0.122772 * np.cos(2 * np.pi * NDay / 366 + 1.498311)
                      - 0.165458 * np.cos(4 * np.pi * NDay / 366 - 1.261546)
                      - 0.005354 * np.cos(6 * np.pi * NDay / 366 - 1.1571)   )
    return eed

## A.4 時角の計算 (仕様書6.2 式(7))

- 時角$T_{d,t}[deg]$, 時刻$t[hour]$, 均時差$e_d[hour]$, 経度$L[deg]$

$$T_{d,t} = (t + e_d - 12) \times 15 + (L - 135)\qquad (7) $$

In [7]:
def calc_Tdt(Longitude, eed, TT):
    
    Tdt = ( TT + eed - 12) * 15 +(Longitude - 135)
    
    return Tdt

## A.5 太陽高度の正弦の計算 (仕様書6.2 式(8)

- 太陽高度$h_{S,d,t}[deg]$, 緯度$\phi[deg]$, 赤緯$\delta_d[deg]$,時角$T_{d,t}[deg]$

$$\sin h_{S,d,t} = max[0, \sin \phi \sin \delta_d + \cos \phi \cos \delta_d \cos T_{d,t}] \qquad (8) $$

In [8]:
def calc_sinh(Latitude, deltad, Tdt):
    
    sinh = max(0, 
               np.sin(np.radians(Latitude)) * np.sin(np.radians(deltad)) 
                 + np.cos(np.radians(Latitude)) * np.cos(np.radians(deltad)) * np.cos(np.radians(Tdt)) )
   
    return sinh

## A.6 太陽高度とその余弦の計算 (仕様書6.2 式(9))

- 太陽高度$h_{S,d,t}[deg]$

$$\cos h_{S,d,t} = (1 - \sin ^2 h_{S,d,t})^{0.5} \qquad (9) $$

- 式(8), (9)より、$h_{S,d,t} = \tan^{-1} (\sin h_{S,d,t} / \cos h_{S,d,t})$

In [9]:
def calc_cosh(sinh):
    
    cosh = (1 - sinh**2) **0.5
   
    return cosh

In [10]:
def calc_hsdt(cosh, sinh):
    
    hsdt = np.rad2deg(np.arctan( sinh / cosh ))
    
    return hsdt

## A.7 太陽方位角の計算 (仕様書6.2 式(10)～(12))

- 太陽方位角$A_{ZS,d,t}[deg]$, 太陽高度$h_{S,d,t}[deg]$, 赤緯$\delta_d[deg]$, 時角$T_{d,t}[deg]$, 緯度$\phi[deg]$

$$\sin A_{ZS,d,t} = \cos \delta_d \sin T_{d,t} / \cos h_{S,d,t} \qquad (10) $$

$$\cos A_{ZS,d,t} = (\sin h_{S,d,t} \sin \phi - \sin \delta_d) / (\cos h_{S,d,t} \cos \phi) \qquad (11) $$

$$
A_{ZS,d,t} = \left\{
\begin{array}{ll}
\tan^{-1} (\sin A_{ZS,d,t} / \cos A_{ZS,d,t}) + 180 \hspace{24pt} (\sin A_{ZS,d,t} > 0, \cos A_{ZS,d,t} < 0)
\\
\tan^{-1} (\sin A_{ZS,d,t} / \cos A_{ZS,d,t}) - 180 \hspace{24pt} (\sin A_{ZS,d,t} < 0, \cos A_{ZS,d,t} < 0)
\\
90 \hspace{136pt} (\sin A_{ZS,d,t} = 1, \cos A_{ZS,d,t} = 0)
\\
-90 \hspace{130pt} (\sin A_{ZS,d,t} = -1, \cos A_{ZS,d,t} = 0)
\\
\tan^{-1} (\sin A_{ZS,d,t} / \cos A_{ZS,d,t}) \hspace{48pt} (other)
\end{array}
\right.  \qquad (12) 
$$

In [11]:
def calc_Azsdt(Latitude, deltad, Tdt, sinh, cosh):
    
    sinAzsdt = np.cos(np.radians(deltad)) * np.sin(np.radians(Tdt)) / cosh
    cosAzsdt = ( ( sinh * np.sin(np.radians(Latitude)) - np.sin(np.radians(deltad)) ) 
             /   ( cosh * np.cos(np.radians(Latitude)) ) )
    if abs(sinAzsdt) == 1:
        Azsdt = 90 * sinAzsdt
    elif sinAzsdt > 0 and cosAzsdt < 0:
        Azsdt = np.rad2deg(np.arctan( sinAzsdt / cosAzsdt )) + 180
    elif sinAzsdt < 0 and cosAzsdt < 0:        
        Azsdt = np.rad2deg(np.arctan( sinAzsdt / cosAzsdt) ) - 180   
    else:
        Azsdt = np.rad2deg(np.arctan( sinAzsdt / cosAzsdt) )
   
    return Azsdt